In [3]:
import cv2
import numpy as np
import fitz
import os

In [11]:
def pdf_2_jpg(src_path,data_path,file_name):
    # PDF to JPG
    doc = fitz.open(src_path+file_name+'.pdf')
        
    #iterate through the pages of the document and create a RGB image of the page
    for i in range(len(doc)):
        pix = doc[i].get_pixmap()
        pix.save(data_path+file_name+'/'+'page'+ str(i) +'.jpg')


src_path = '/home/ppavithra/Desktop/notebook/Opencv-main/User-001/'
data_path = '/home/ppavithra/Desktop/notebook/Opencv-main/User-001/'
#files = [file for file in os.listdir(src_path) if '.pdf' in file]
files = ['User-005.pdf']

for file_name in files:
    file_name = file_name.rsplit('.',1)[0]
    try:
        os.makedirs(data_path+file_name)
    except:
        print('folder already exist')

    try:
        os.makedirs(data_path+file_name+'/identified_box')
    except:
        print('folder already exist')

    pdf_2_jpg(src_path,data_path,file_name)

    ls = [file for file in os.listdir(data_path+file_name) if 'page' in file]
    ls = sorted(ls)
    for i in ls:
        img = cv2.imread(data_path+file_name+'/'+i)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ret,thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
        contours,hierarchy=cv2.findContours(thresh, cv2.RETR_LIST,
                                    cv2.CHAIN_APPROX_NONE)

        for cnt in contours :
            area = cv2.contourArea(cnt)
            # Shortlisting the regions based on there area.
            approx = cv2.approxPolyDP(cnt, 0.1 * cv2.arcLength(cnt, True), True)
            
            # Checking if the no. of sides of the selected region is 4.
            if (len(approx) == 4) and area>100 and area<35000:
                cv2.drawContours(img, [approx], 0, (0, 255, 0), 1)
            if (len(approx)<4) and area>1000 and area<35000:
                (x, y, w, h) = cv2.boundingRect(approx)
                cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),1)
                
        cv2.imwrite(data_path+file_name+'/identified_box/'+i, img)


In [13]:
!pip install pytesseract

  Using cached pytesseract-0.3.10-py3-none-any.whl (14 kB)


In [19]:
import io
import cv2
import numpy as np
import pytesseract
from PIL import Image
from pytesseract import Output
from wand.image import Image as wi
import sys


pdf = wi(filename="doc.pdf", resolution=100)
pdfImg = pdf.convert('jpg')
j = 1
imgBlobs = []
img1= []
for img in pdfImg.sequence:
    page = wi(image=img)
    page.save(filename=str(j)+".jpg")
    img1.append(cv2.imread(str(j)+".jpg"))
    j += 1

extracted_text = []

for img2 in img1:
    d = pytesseract.image_to_data(img2, output_type=Output.DICT)
    n_boxes = len(d['level'])
    print(n_boxes)
    extracted_text.append(d['text'][9])
    (x, y, w, h) = (d['left'][9], d['top'][9], d['width'][9], d['height'][9])
    cv2.rectangle(img2, (x, y), (x + w, y + h), (0, 255, 0), 2)


    cv2.imshow('img', img2)

    print(d)


with open('Prototype.txt', 'w') as filehandle:
        for listitem in extracted_text:
            filehandle.write('%s\n' % listitem)

PolicyError: attempt to perform an operation not allowed by the security policy `PDF' @ error/constitute.c/IsCoderAuthorized/408